In [316]:
import numpy as np
import pandas as pd
import matplotlib

import matplotlib.pyplot as plt
%matplotlib inline
import os
import warnings
import seaborn as sns
import plotly.graph_objs as go
import plotly.figure_factory as ff
import plotly.express as px

from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.model_selection import train_test_split

## I. Data transformation step

* Tính tỷ lệ % số lệnh giao dịch hoặc giá trị giao dịch theo kênh / grouped kênh lại thành 3 nhóm: API, web và mobile. Nếu có API giao dịch thì flag lên riêng. >> Done
* Điều chỉnh data tháng về định dạng ngày để làm datediff dễ hơn >> Done
* Max NAV, NAV gần nhất, MAX cash balance, cash balance gần nhất >> Done

* tỷ lệ NAV hiện tại trên max NAV và tỷ lệ cash hiện tại trên max Cash >> Done
* NAV của Khách hàng bao gồm cả số dư tiền =>> Có thể bổ sung thêm phân tích tỷ lệ tiền / NAV của Khách hàng để thể hiện việc Khách hàng có đang idle và đợi hay không? >> Done

* Trend của các chỉ số về NAV, cash, Trading này qua moving averages 3 tháng gần nhất tại từng tháng và giá trị % của số hiện tại so với moving averages. >> Done

* Số lượng cổ phiếu giao dịch tổng cộng và theo mỗi tháng - mua càng nhiều cổ thì càng là active trader, groupby customer_code và tháng mua của Khách hàng >> Done

* Join thêm ngày mở vào data và thêm month-on-board bằng datediff để làm cohort analysis.

ITERATION 2:
* Top 2 ngành KH giao dịch nhiều nhất, top 3 mã cổ phiếu giao dịch nhiều nhất trong suốt thời gian giao dịch

### tính tỷ lệ % số lệnh giao dịch hoặc giá trị giao dịch theo kênh / grouped kênh lại thành 3 nhóm: API, web và mobile. Nếu có API giao dịch thì flag lên riêng. >> Done

In [317]:
# Get the channel dataset
Trading_by_Channel = pd.read_excel("Churn analysis _ Trading by Channel_done.xlsx") 

In [318]:
# Check what is the head of the dataset
Trading_by_Channel.head()

,CHANNEL_MAPPING,Unknown,Fast Connect API,iBoard,iBoard App,iBoard Web,Mobile Trading,OEF Online,Pro Trading,Web Trading
0,000877FD1B07EB1646272C0BF682B5,NaN,NaN,4.826250e+06,NaN,NaN,NaN,NaN,NaN,NaN
1,0020100DBC05FEB6E5FAAE2D0FF6C5,NaN,NaN,1.717045e+09,NaN,NaN,NaN,NaN,NaN,NaN
2,0024BB102F2A8FAB9310636215AC7D,NaN,NaN,5.715500e+05,NaN,NaN,NaN,NaN,NaN,NaN
3,00342E84644844DC10A7D75ED704E7,NaN,NaN,8.700000e+06,NaN,NaN,NaN,NaN,NaN,NaN
4,004228781AB402B85BFCF45A44DF08,NaN,NaN,5.860000e+04,NaN,NaN,NaN,NaN,NaN,NaN


In [319]:
Trading_by_Channel.columns

Index(['CHANNEL_MAPPING', 'Unknown', 'Fast Connect API', 'iBoard',
       'iBoard App', 'iBoard Web', 'Mobile Trading', 'OEF Online',
       'Pro Trading', 'Web Trading'],
      dtype='object')

In [320]:
Trading_by_Channel['Total'] = Trading_by_Channel.sum(axis=1)

C:\Users\DUONG\AppData\Local\Temp\ipykernel_9328\1864070090.py:1: FutureWarning:

Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.



In [321]:
Trading_by_Channel['Total'].head()

0    4.826250e+06
1    1.717045e+09
2    5.715500e+05
3    8.700000e+06
4    5.860000e+04
Name: Total, dtype: float64

In [322]:
web_trading = ['iBoard Web', 'Pro Trading', 'Web Trading' ]
mobile_trading = ['iBoard', 'iBoard App', 'Mobile Trading']
others = ['Unknown', 'OEF Online']
api = ['Fast Connect API']

Trading_by_Channel['web_trading'] = Trading_by_Channel[web_trading].sum(axis=1)
Trading_by_Channel['mobile_trading'] = Trading_by_Channel[mobile_trading].sum(axis=1)
Trading_by_Channel['others'] = Trading_by_Channel[others].sum(axis=1)
Trading_by_Channel['api'] = Trading_by_Channel[api].sum(axis=1)

In [323]:
Trading_by_Channel.head()

,CHANNEL_MAPPING,Unknown,Fast Connect API,iBoard,iBoard App,iBoard Web,Mobile Trading,OEF Online,Pro Trading,Web Trading,Total,web_trading,mobile_trading,others,api
0,000877FD1B07EB1646272C0BF682B5,NaN,NaN,4.826250e+06,NaN,NaN,NaN,NaN,NaN,NaN,4.826250e+06,0.0,4.826250e+06,0.0,0.0
1,0020100DBC05FEB6E5FAAE2D0FF6C5,NaN,NaN,1.717045e+09,NaN,NaN,NaN,NaN,NaN,NaN,1.717045e+09,0.0,1.717045e+09,0.0,0.0
2,0024BB102F2A8FAB9310636215AC7D,NaN,NaN,5.715500e+05,NaN,NaN,NaN,NaN,NaN,NaN,5.715500e+05,0.0,5.715500e+05,0.0,0.0
3,00342E84644844DC10A7D75ED704E7,NaN,NaN,8.700000e+06,NaN,NaN,NaN,NaN,NaN,NaN,8.700000e+06,0.0,8.700000e+06,0.0,0.0
4,004228781AB402B85BFCF45A44DF08,NaN,NaN,5.860000e+04,NaN,NaN,NaN,NaN,NaN,NaN,5.860000e+04,0.0,5.860000e+04,0.0,0.0


In [324]:
Trading_by_Channel['%_web_trade'] = Trading_by_Channel['web_trading'] / Trading_by_Channel['Total']
Trading_by_Channel['%_mobile_trading'] = Trading_by_Channel['mobile_trading'] / Trading_by_Channel['Total']
Trading_by_Channel['%_others'] = Trading_by_Channel['others'] / Trading_by_Channel['Total']
Trading_by_Channel['%_api'] = Trading_by_Channel['api'] / Trading_by_Channel['Total']

In [325]:
Trading_by_Channel.head()

,CHANNEL_MAPPING,Unknown,Fast Connect API,iBoard,iBoard App,iBoard Web,Mobile Trading,OEF Online,Pro Trading,Web Trading,Total,web_trading,mobile_trading,others,api,%_web_trade,%_mobile_trading,%_others,%_api
0,000877FD1B07EB1646272C0BF682B5,NaN,NaN,4.826250e+06,NaN,NaN,NaN,NaN,NaN,NaN,4.826250e+06,0.0,4.826250e+06,0.0,0.0,0.0,1.0,0.0,0.0
1,0020100DBC05FEB6E5FAAE2D0FF6C5,NaN,NaN,1.717045e+09,NaN,NaN,NaN,NaN,NaN,NaN,1.717045e+09,0.0,1.717045e+09,0.0,0.0,0.0,1.0,0.0,0.0
2,0024BB102F2A8FAB9310636215AC7D,NaN,NaN,5.715500e+05,NaN,NaN,NaN,NaN,NaN,NaN,5.715500e+05,0.0,5.715500e+05,0.0,0.0,0.0,1.0,0.0,0.0
3,00342E84644844DC10A7D75ED704E7,NaN,NaN,8.700000e+06,NaN,NaN,NaN,NaN,NaN,NaN,8.700000e+06,0.0,8.700000e+06,0.0,0.0,0.0,1.0,0.0,0.0
4,004228781AB402B85BFCF45A44DF08,NaN,NaN,5.860000e+04,NaN,NaN,NaN,NaN,NaN,NaN,5.860000e+04,0.0,5.860000e+04,0.0,0.0,0.0,1.0,0.0,0.0


In [326]:
Trading_by_Channel = Trading_by_Channel.assign(API_Trade_Flag = np.where(Trading_by_Channel['Fast Connect API'].isnull(),"N","Y"))

In [327]:
Trading_by_Channel.head()

,CHANNEL_MAPPING,Unknown,Fast Connect API,iBoard,iBoard App,iBoard Web,Mobile Trading,OEF Online,Pro Trading,Web Trading,Total,web_trading,mobile_trading,others,api,%_web_trade,%_mobile_trading,%_others,%_api,API_Trade_Flag
0,000877FD1B07EB1646272C0BF682B5,NaN,NaN,4.826250e+06,NaN,NaN,NaN,NaN,NaN,NaN,4.826250e+06,0.0,4.826250e+06,0.0,0.0,0.0,1.0,0.0,0.0,N
1,0020100DBC05FEB6E5FAAE2D0FF6C5,NaN,NaN,1.717045e+09,NaN,NaN,NaN,NaN,NaN,NaN,1.717045e+09,0.0,1.717045e+09,0.0,0.0,0.0,1.0,0.0,0.0,N
2,0024BB102F2A8FAB9310636215AC7D,NaN,NaN,5.715500e+05,NaN,NaN,NaN,NaN,NaN,NaN,5.715500e+05,0.0,5.715500e+05,0.0,0.0,0.0,1.0,0.0,0.0,N
3,00342E84644844DC10A7D75ED704E7,NaN,NaN,8.700000e+06,NaN,NaN,NaN,NaN,NaN,NaN,8.700000e+06,0.0,8.700000e+06,0.0,0.0,0.0,1.0,0.0,0.0,N
4,004228781AB402B85BFCF45A44DF08,NaN,NaN,5.860000e+04,NaN,NaN,NaN,NaN,NaN,NaN,5.860000e+04,0.0,5.860000e+04,0.0,0.0,0.0,1.0,0.0,0.0,N


In [328]:
Trading_by_Channel = Trading_by_Channel.assign(Web_trade_focus_FLG = np.where(Trading_by_Channel['%_web_trade'] >= 0.3,"Y","N"))

In [329]:
fig = px.histogram(Trading_by_Channel, x='%_web_trade')
fig.show()

In [330]:
fig = px.histogram(Trading_by_Channel, x='Web_trade_focus_FLG')
fig.show()

In [331]:
fig = px.histogram(Trading_by_Channel, x='API_Trade_Flag')
fig.show()

### Điều chỉnh data tháng về định dạng ngày để làm datediff dễ hơn >> Done

In [332]:
Transaction_history = pd.read_excel('Churn analysis _ Transaction_history_unpivot.xlsx')
Transaction_history['Month']=Transaction_history['MONTH'].str.replace('T','')
Transaction_history = Transaction_history.rename(columns={'YEAR': 'Year'})
# Transaction_history['Year'].astype('int')
# Transaction_history['Month'].astype('int')
Transaction_history['Day'] = '1'
# Transaction_history['Day'].astype('int')
Transaction_history['Date'] = pd.to_datetime(Transaction_history[['Year', 'Month', 'Day']])
Transaction_history['Date'].head()

0   2022-11-01
1   2022-11-01
2   2022-11-01
3   2022-11-01
4   2022-12-01
Name: Date, dtype: datetime64[ns]

In [333]:
Transaction_history.head()

,CUSTOMER_CODE,Year,MONTH,ATTRIBUTE,Value,Month,Day,Date
0,0000F727854B50BB95C054B39C1FE5,2022,T11,Avg NAV Monthly,500001.294117,11,1,2022-11-01
1,0000F727854B50BB95C054B39C1FE5,2022,T11,First CUSTOMER_STATUS,Newly Opened,11,1,2022-11-01
2,0000F727854B50BB95C054B39C1FE5,2022,T11,Avg Cash Balance,500001.294118,11,1,2022-11-01
3,0000F727854B50BB95C054B39C1FE5,2022,T11,Net Cash,500000,11,1,2022-11-01
4,0000F727854B50BB95C054B39C1FE5,2022,T12,Avg NAV Monthly,492823.318181,12,1,2022-12-01


### Max NAV, NAV gần nhất, MAX cash balance, cash balance gần nhất, tỷ lệ NAV hiện tại trên max NAV, tỷ lệ cash hiện tại trên max Cash, tỷ lệ cash trên NAV trong từng giai đoạn.

In [334]:
Transaction_history_NAV = Transaction_history[Transaction_history['ATTRIBUTE'] == 'Avg NAV Monthly']
Transaction_history_Cash = Transaction_history[Transaction_history['ATTRIBUTE'] == 'Avg Cash Balance']
Transaction_history_NAV = Transaction_history_NAV.rename(columns={'Value':'NAV'})
Transaction_history_Cash = Transaction_history_Cash.rename(columns={'Value':'Cash'})


In [335]:
# Tính max NAV và max Cash theo từng Khách hàng 
Transaction_history_NAV_max = Transaction_history_NAV.groupby(['CUSTOMER_CODE']).agg({'NAV':'max'})
Transaction_history_NAV_max = Transaction_history_NAV_max.reset_index()
Transaction_history_Cash_max =Transaction_history_Cash.groupby(['CUSTOMER_CODE']).agg({'Cash':'max'})
Transaction_history_Cash_max = Transaction_history_Cash_max.reset_index()

In [336]:
Transaction_history_Cash_max.columns

Index(['CUSTOMER_CODE', 'Cash'], dtype='object')

In [337]:
Transaction_history_Cash_max = Transaction_history_Cash_max.rename(columns={'Cash': 'Max_Cash'})
Transaction_history_NAV_max = Transaction_history_NAV_max.rename(columns={'NAV': 'Max_NAV'})

In [338]:
Transaction_history_NAV_max.head()
Transaction_history_Cash_max.head()

,CUSTOMER_CODE,Max_Cash
0,0000F727854B50BB95C054B39C1FE5,500001.294118
1,00011FF69DC01DA5E085BC4B88FBCC,0.000000
2,000217E11EE25BDC5B522246401A7D,0.000000
3,0002511664BFF302E40B19ED6872AD,0.000000
4,0005212086E5708F5E2A596558BA5F,0.000000


In [339]:
test1 = Transaction_history_NAV[Transaction_history_NAV['CUSTOMER_CODE'] == '0000F727854B50BB95C054B39C1FE5']
test1

,CUSTOMER_CODE,Year,MONTH,ATTRIBUTE,NAV,Month,Day,Date
0,0000F727854B50BB95C054B39C1FE5,2022,T11,Avg NAV Monthly,500001.294117,11,1,2022-11-01
4,0000F727854B50BB95C054B39C1FE5,2022,T12,Avg NAV Monthly,492823.318181,12,1,2022-12-01
7,0000F727854B50BB95C054B39C1FE5,2023,T1,Avg NAV Monthly,484652.8125,1,1,2023-01-01
10,0000F727854B50BB95C054B39C1FE5,2023,T2,Avg NAV Monthly,475441.35,2,1,2023-02-01
13,0000F727854B50BB95C054B39C1FE5,2023,T3,Avg NAV Monthly,466438.695652,3,1,2023-03-01
16,0000F727854B50BB95C054B39C1FE5,2023,T4,Avg NAV Monthly,457897.4,4,1,2023-04-01
19,0000F727854B50BB95C054B39C1FE5,2023,T5,Avg NAV Monthly,449125.45,5,1,2023-05-01


In [340]:
test2 = Transaction_history_NAV_max[Transaction_history_NAV_max['CUSTOMER_CODE'] == '0000F727854B50BB95C054B39C1FE5']
test2

,CUSTOMER_CODE,Max_NAV
0,0000F727854B50BB95C054B39C1FE5,500001.294117


In [341]:
# Get the latest value 
Transaction_history_NAV_sorted = Transaction_history_NAV.sort_values(by='Date') 
Transaction_history_NAV_Latest = Transaction_history_NAV_sorted.drop_duplicates('CUSTOMER_CODE', keep ='last')
Transaction_history_NAV_Latest = Transaction_history_NAV_Latest.rename(columns={'NAV': 'Latest_NAV'})
Transaction_history_Cash_sorted = Transaction_history_Cash.sort_values(by='Date') 
Transaction_history_Cash_Latest = Transaction_history_Cash_sorted.drop_duplicates('CUSTOMER_CODE', keep ='last')
Transaction_history_Cash_Latest = Transaction_history_Cash_Latest.rename(columns={'Cash': 'Latest_Cash'})

In [342]:
# Group the above into one data frame and calculate the neccessary measures:
# You can first access the relevant dataframe columns via their labels (e.g. df_a[['EntityNum', 'foo']] and then join those.
# df_a[['EntityNum', 'foo']].merge(df_b[['EntityNum', 'a_col']], on='EntityNum', how='left')

# Tables to be joined: 
# Transaction_history_NAV_Latest
# Transaction_history_Cash_Latest
# Transaction_history_NAV
# Transaction_history_Cash
# Transaction_history_NAV_max
# Transaction_history_Cash_max

# Join is done on CUSTOMER_CODE column and get the values columns from each dataframe 

In [343]:
Transaction_history_NAV.head()

,CUSTOMER_CODE,Year,MONTH,ATTRIBUTE,NAV,Month,Day,Date
0,0000F727854B50BB95C054B39C1FE5,2022,T11,Avg NAV Monthly,500001.294117,11,1,2022-11-01
4,0000F727854B50BB95C054B39C1FE5,2022,T12,Avg NAV Monthly,492823.318181,12,1,2022-12-01
7,0000F727854B50BB95C054B39C1FE5,2023,T1,Avg NAV Monthly,484652.8125,1,1,2023-01-01
10,0000F727854B50BB95C054B39C1FE5,2023,T2,Avg NAV Monthly,475441.35,2,1,2023-02-01
13,0000F727854B50BB95C054B39C1FE5,2023,T3,Avg NAV Monthly,466438.695652,3,1,2023-03-01


In [344]:
Transaction_history_Cash.head()

,CUSTOMER_CODE,Year,MONTH,ATTRIBUTE,Cash,Month,Day,Date
2,0000F727854B50BB95C054B39C1FE5,2022,T11,Avg Cash Balance,500001.294118,11,1,2022-11-01
6,0000F727854B50BB95C054B39C1FE5,2022,T12,Avg Cash Balance,492823.318182,12,1,2022-12-01
9,0000F727854B50BB95C054B39C1FE5,2023,T1,Avg Cash Balance,484652.8125,1,1,2023-01-01
12,0000F727854B50BB95C054B39C1FE5,2023,T2,Avg Cash Balance,475441.35,2,1,2023-02-01
15,0000F727854B50BB95C054B39C1FE5,2023,T3,Avg Cash Balance,466438.695652,3,1,2023-03-01


In [345]:
Historical_details_1 = Transaction_history_NAV[['CUSTOMER_CODE','Date','NAV']].merge(Transaction_history_Cash[['CUSTOMER_CODE','Date','Cash']], how='left', left_on=['CUSTOMER_CODE','Date'], right_on = ['CUSTOMER_CODE','Date'])
Historical_details_max_NAV = Historical_details_1.merge(Transaction_history_NAV_max, how='left', left_on='CUSTOMER_CODE', right_on = 'CUSTOMER_CODE')
Historical_details_max_Cash = Historical_details_max_NAV.merge(Transaction_history_Cash_max, how='left', left_on='CUSTOMER_CODE', right_on = 'CUSTOMER_CODE')
Historical_details_latest_Cash = Historical_details_max_Cash.merge(Transaction_history_Cash_Latest[['CUSTOMER_CODE','Latest_Cash']], how='left', left_on='CUSTOMER_CODE', right_on = 'CUSTOMER_CODE')
Historical_details_final = Historical_details_latest_Cash.merge(Transaction_history_NAV_Latest[['CUSTOMER_CODE','Latest_NAV']], how='left', left_on='CUSTOMER_CODE', right_on = 'CUSTOMER_CODE')
Historical_details_final.head()

,CUSTOMER_CODE,Date,NAV,Cash,Max_NAV,Max_Cash,Latest_Cash,Latest_NAV
0,0000F727854B50BB95C054B39C1FE5,2022-11-01,500001.294117,500001.294118,500001.294117,500001.294118,449125.45,449125.45
1,0000F727854B50BB95C054B39C1FE5,2022-12-01,492823.318181,492823.318182,500001.294117,500001.294118,449125.45,449125.45
2,0000F727854B50BB95C054B39C1FE5,2023-01-01,484652.8125,484652.8125,500001.294117,500001.294118,449125.45,449125.45
3,0000F727854B50BB95C054B39C1FE5,2023-02-01,475441.35,475441.35,500001.294117,500001.294118,449125.45,449125.45
4,0000F727854B50BB95C054B39C1FE5,2023-03-01,466438.695652,466438.695652,500001.294117,500001.294118,449125.45,449125.45


In [346]:
Historical_details_final['%_cash_over_max_cash'] = np.divide(Historical_details_final['Cash'],  Historical_details_final['Max_Cash'], where = Historical_details_final['Max_Cash'] != 0)
Historical_details_final['%_NAV_over_max_NAV'] = np.divide(Historical_details_final['NAV'],  Historical_details_final['Max_NAV'], where = Historical_details_final['Max_NAV'] != 0)
Historical_details_final['%_Latest_Cash_over_max_cash'] = np.divide(Historical_details_final['Latest_Cash'],  Historical_details_final['Max_Cash'], where = Historical_details_final['Max_Cash'] != 0)
Historical_details_final['%_Latest_NAV_over_max_NAV'] = np.divide(Historical_details_final['Latest_NAV'],  Historical_details_final['Max_NAV'], where = Historical_details_final['Max_NAV'] != 0)
Historical_details_final['%_Cash_over_NAV'] = np.divide(Historical_details_final['Cash'],  Historical_details_final['NAV'], where = Historical_details_final['NAV'] != 0)

###  Trend của các chỉ số về NAV, cash, Trading này qua moving averages 3 tháng gần nhất tại từng tháng và giá trị % của số hiện tại so với moving averages.

In [347]:
Historical_details_final['MA_3M_NAV'] = Historical_details_final.groupby('CUSTOMER_CODE').rolling(3)['NAV'].mean().reset_index(drop=True)
Historical_details_final['MA_3M_Cash'] = Historical_details_final.groupby('CUSTOMER_CODE').rolling(3)['Cash'].mean().reset_index(drop=True)
Historical_details_final['%NAV_MA_3M_NAV'] = np.divide(Historical_details_final['NAV'],  Historical_details_final['MA_3M_NAV'], where = Historical_details_final['MA_3M_NAV'] != 0)
Historical_details_final['%Cash_MA_3M_Cash'] = np.divide(Historical_details_final['Cash'],  Historical_details_final['MA_3M_Cash'], where = Historical_details_final['MA_3M_Cash'] != 0)

### Số lượng cổ phiếu giao dịch tổng cộng và theo mỗi tháng - mua càng nhiều cổ thì càng là active trader, groupby customer_code và tháng mua của Khách hàng

In [348]:
Trading_history = pd.read_excel('Churn analysis _ Trading history_unpivot.xlsx')

In [349]:
Trading_history.columns

Index(['CUSTOMER_CODE', 'Stock', 'STOCK_TYPE', 'Attributes', 'Value', 'Year',
       'Month', 'Day', 'Date'],
      dtype='object')

In [350]:
Trading_history_orders = Trading_history[Trading_history['Attributes'] == 'No. of orders']
Trading_history_value = Trading_history[Trading_history['Attributes'] == 'Matched Value']

In [351]:
Trading_history_orders = Trading_history_orders.rename(columns={'Value':'No_orders'})
Trading_history_value = Trading_history_value.rename(columns={'Value':'Matched_value'})

In [352]:
Trading_history_processed = Trading_history_orders.drop(labels='Attributes',axis=1).merge(Trading_history_value[['CUSTOMER_CODE','Stock','Date','Matched_value']],how='left', left_on=['CUSTOMER_CODE','Stock','Date'],right_on=['CUSTOMER_CODE','Stock','Date'])

In [353]:
Trading_history_processed.head()

,CUSTOMER_CODE,Stock,STOCK_TYPE,No_orders,Year,Month,Day,Date,Matched_value
0,000877FD1B07EB1646272C0BF682B5,HPG,Thép và sản phẩm thép,1,2022,10,1,2022-10-01,313000.0
1,000877FD1B07EB1646272C0BF682B5,HPG,Thép và sản phẩm thép,2,2022,11,1,2022-11-01,88650.0
2,000877FD1B07EB1646272C0BF682B5,HPG,Thép và sản phẩm thép,1,2022,12,1,2022-12-01,388000.0
3,000877FD1B07EB1646272C0BF682B5,LDG,"Tư Vấn, Định giá, Môi giới Bất động sản",6,2022,11,1,2022-11-01,2437000.0
4,000877FD1B07EB1646272C0BF682B5,PIT,Bán lẻ phức hợp,3,2022,11,1,2022-11-01,510000.0


In [354]:
## Group by dùng nhiều cột theo format
## f = {'A':['sum','mean'], 'B':['prod'], 'D': {'my name': cust}}
## df.groupby('GRP').agg(f)

In [355]:
Trading_history_processed.columns

Index(['CUSTOMER_CODE', 'Stock', 'STOCK_TYPE', 'No_orders', 'Year', 'Month',
       'Day', 'Date', 'Matched_value'],
      dtype='object')

In [356]:
f = {'Stock':['nunique'], 'Matched_value':['sum'], 'No_orders':['sum']}
Trading_history_pivot_date = Trading_history_processed.groupby(['CUSTOMER_CODE','Date']).agg(f).reset_index()
Trading_history_pivot_customer = Trading_history_processed.groupby(['CUSTOMER_CODE'])['Stock'].nunique().reset_index()

In [357]:
Trading_history_pivot_date.columns = ['_'.join(col) for col in Trading_history_pivot_date.columns.values]
Trading_history_pivot_date.rename(columns={'CUSTOMER_CODE_':'CUSTOMER_CODE','Date_':'Date'})
Trading_history_pivot_date.head()

,CUSTOMER_CODE_,Date_,Stock_nunique,Matched_value_sum,No_orders_sum
0,000877FD1B07EB1646272C0BF682B5,2022-10-01,2,542000.0,2
1,000877FD1B07EB1646272C0BF682B5,2022-11-01,4,3108550.0,14
2,000877FD1B07EB1646272C0BF682B5,2022-12-01,3,1175700.0,3
3,0020100DBC05FEB6E5FAAE2D0FF6C5,2022-10-01,1,4960000.0,1
4,0020100DBC05FEB6E5FAAE2D0FF6C5,2022-11-01,3,50635000.0,14


In [358]:
Trading_history_pivot_date.columns

Index(['CUSTOMER_CODE_', 'Date_', 'Stock_nunique', 'Matched_value_sum',
       'No_orders_sum'],
      dtype='object')

In [359]:
Trading_history_pivot_date.rename(columns={'CUSTOMER_CODE_':'CUSTOMER_CODE'})
Trading_history_pivot_date.rename(columns={'Date_':'Date'})
Trading_history_pivot_date.head()

,CUSTOMER_CODE_,Date_,Stock_nunique,Matched_value_sum,No_orders_sum
0,000877FD1B07EB1646272C0BF682B5,2022-10-01,2,542000.0,2
1,000877FD1B07EB1646272C0BF682B5,2022-11-01,4,3108550.0,14
2,000877FD1B07EB1646272C0BF682B5,2022-12-01,3,1175700.0,3
3,0020100DBC05FEB6E5FAAE2D0FF6C5,2022-10-01,1,4960000.0,1
4,0020100DBC05FEB6E5FAAE2D0FF6C5,2022-11-01,3,50635000.0,14


In [360]:
Trading_history_pivot_customer.head()

,CUSTOMER_CODE,Stock
0,000877FD1B07EB1646272C0BF682B5,4
1,0020100DBC05FEB6E5FAAE2D0FF6C5,22
2,0024BB102F2A8FAB9310636215AC7D,1
3,00342E84644844DC10A7D75ED704E7,1
4,004228781AB402B85BFCF45A44DF08,1


In [361]:
Trading_history_pivot_date.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21079 entries, 0 to 21078
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   CUSTOMER_CODE_     21079 non-null  object        
 1   Date_              21079 non-null  datetime64[ns]
 2   Stock_nunique      21079 non-null  int64         
 3   Matched_value_sum  21079 non-null  float64       
 4   No_orders_sum      21079 non-null  int64         
dtypes: datetime64[ns](1), float64(1), int64(2), object(1)
memory usage: 823.5+ KB


In [362]:
Historical_details_final = Historical_details_final.merge(Trading_history_pivot_date, how = 'left', left_on = ['CUSTOMER_CODE','Date'], right_on = ['CUSTOMER_CODE_','Date_'])

In [363]:
Historical_details_final = Historical_details_final.drop(['CUSTOMER_CODE_','Date_'], axis=1)

### Join thêm ngày mở vào data và thêm month-on-board bằng datediff để làm cohort analysis.

In [364]:
Customer_Demographic = pd.read_excel('Churn analysis _ Demographic.xlsx')

In [365]:
Customer_Demographic.head()

,CUSTOMER_CODE,AGE,CUSTOMER_PRODUCT_TYPE,WEALTH_RANGE_NAME,CUSTOMER STATUS,Open_Date
0,0000F727854B50BB95C054B39C1FE5,42,Not yet trading,W2:(0-10Mn),Dummy,2022-11-08
1,00342E84644844DC10A7D75ED704E7,29,Stock Only,W5:(1Bn-2Bn),Engaged,2022-12-15
2,00350176191BA363DCA9042B3F11F2,26,Not yet trading,W1:(<=0Mn),Newly Opened,2023-06-12
3,00663E5CE35AA4E3834FCF00084804,33,Not yet trading,W2:(0-10Mn),Newly Opened,2023-05-17
4,02AC0F84428DFD0D42B4E984DCAF4A,22,Stock Only,W2:(0-10Mn),Churn,2022-12-14


In [366]:
Historical_details_final = Historical_details_final.merge(Customer_Demographic, how='left', left_on= 'CUSTOMER_CODE', right_on = 'CUSTOMER_CODE')

In [367]:
Historical_details_final['Month_on_board'] = ((Historical_details_final.Date -  Historical_details_final.Open_Date)/np.timedelta64(1, 'M')) + 1

## II. EDA:
* Nên có 2 view: (1) Sum hết data vào một chỗ để nhìn demographic của Khách hàng (2) Merge demographic vào bảng giao dịch của KH để tìm pattern về giao dịch 
* Pick thử một vài mẫu dữ liệu cụ thể và nhìn trend của các mẫu này qua thời gian của từng dữ liệu, dùng như kiểu heatmap để đưa ra giả định về khách hàng ( ví dụ như trend ngày trễ của Khách hàng đi như thế nào? >> trend giao dịch qua thời gian dạng cohort sẽ đi như thế nào? )

In [368]:
Historical_details_final = Historical_details_final[Historical_details_final['Open_Date'].notnull()]

In [369]:
Historical_details_final[Historical_details_final['Month_on_board'] < 0] = 1

In [370]:
Historical_details_final['Month_on_board'] = Historical_details_final['Month_on_board'].apply(np.ceil)


In [371]:
Trading_by_Channel.columns

Index(['CHANNEL_MAPPING', 'Unknown', 'Fast Connect API', 'iBoard',
       'iBoard App', 'iBoard Web', 'Mobile Trading', 'OEF Online',
       'Pro Trading', 'Web Trading', 'Total', 'web_trading', 'mobile_trading',
       'others', 'api', '%_web_trade', '%_mobile_trading', '%_others', '%_api',
       'API_Trade_Flag', 'Web_trade_focus_FLG'],
      dtype='object')

In [372]:
Historical_details_final = Historical_details_final.merge(Trading_by_Channel[['CHANNEL_MAPPING','Web_trade_focus_FLG']], how='left', left_on = 'CUSTOMER_CODE', right_on = 'CHANNEL_MAPPING')

In [373]:
Historical_details_final = Historical_details_final.drop(labels='CHANNEL_MAPPING', axis=1)

In [374]:
Historical_details_final.head(20)

,CUSTOMER_CODE,Date,NAV,Cash,Max_NAV,Max_Cash,Latest_Cash,Latest_NAV,%_cash_over_max_cash,%_NAV_over_max_NAV,...,Stock_nunique,Matched_value_sum,No_orders_sum,AGE,CUSTOMER_PRODUCT_TYPE,WEALTH_RANGE_NAME,CUSTOMER STATUS,Open_Date,Month_on_board,Web_trade_focus_FLG
0,0000F727854B50BB95C054B39C1FE5,2022-11-01 00:00:00,500001.294117,500001.294118,5.000013e+05,5.000013e+05,449125.45,449125.45,1.0,1.0,...,NaN,NaN,NaN,42.0,Not yet trading,W2:(0-10Mn),Dummy,2022-11-08 00:00:00,1.0,NaN
1,0000F727854B50BB95C054B39C1FE5,2022-12-01 00:00:00,492823.318181,492823.318182,5.000013e+05,5.000013e+05,449125.45,449125.45,0.985644,0.985644,...,NaN,NaN,NaN,42.0,Not yet trading,W2:(0-10Mn),Dummy,2022-11-08 00:00:00,2.0,NaN
2,0000F727854B50BB95C054B39C1FE5,2023-01-01 00:00:00,484652.8125,484652.8125,5.000013e+05,5.000013e+05,449125.45,449125.45,0.969303,0.969303,...,NaN,NaN,NaN,42.0,Not yet trading,W2:(0-10Mn),Dummy,2022-11-08 00:00:00,3.0,NaN
3,0000F727854B50BB95C054B39C1FE5,2023-02-01 00:00:00,475441.35,475441.35,5.000013e+05,5.000013e+05,449125.45,449125.45,0.95088,0.95088,...,NaN,NaN,NaN,42.0,Not yet trading,W2:(0-10Mn),Dummy,2022-11-08 00:00:00,4.0,NaN
4,0000F727854B50BB95C054B39C1FE5,2023-03-01 00:00:00,466438.695652,466438.695652,5.000013e+05,5.000013e+05,449125.45,449125.45,0.932875,0.932875,...,NaN,NaN,NaN,42.0,Not yet trading,W2:(0-10Mn),Dummy,2022-11-08 00:00:00,5.0,NaN
5,0000F727854B50BB95C054B39C1FE5,2023-04-01 00:00:00,457897.4,457897.4,5.000013e+05,5.000013e+05,449125.45,449125.45,0.915792,0.915792,...,NaN,NaN,NaN,42.0,Not yet trading,W2:(0-10Mn),Dummy,2022-11-08 00:00:00,6.0,NaN
6,0000F727854B50BB95C054B39C1FE5,2023-05-01 00:00:00,449125.45,449125.45,5.000013e+05,5.000013e+05,449125.45,449125.45,0.898249,0.898249,...,NaN,NaN,NaN,42.0,Not yet trading,W2:(0-10Mn),Dummy,2022-11-08 00:00:00,7.0,NaN
7,000877FD1B07EB1646272C0BF682B5,2022-10-01 00:00:00,1100098.5,1100005.5,1.822032e+06,1.100006e+06,27,27,1.0,0.603776,...,2.0,542000.0,2.0,22.0,Stock Only,W2:(0-10Mn),Churn,2022-10-28 00:00:00,1.0,N
8,000877FD1B07EB1646272C0BF682B5,2022-11-01 00:00:00,1822032.272727,567278.818182,1.822032e+06,1.100006e+06,27,27,0.515705,1.0,...,4.0,3108550.0,14.0,22.0,Stock Only,W2:(0-10Mn),Churn,2022-10-28 00:00:00,2.0,N
9,000877FD1B07EB1646272C0BF682B5,2022-12-01 00:00:00,435297.272727,329201.318182,1.822032e+06,1.100006e+06,27,27,0.299272,0.238908,...,3.0,1175700.0,3.0,22.0,Stock Only,W2:(0-10Mn),Churn,2022-10-28 00:00:00,3.0,N


In [375]:
Transaction_history_Net_cash = Transaction_history[Transaction_history['ATTRIBUTE'] == 'Net Cash']

In [376]:
Transaction_history_Net_cash.head()

,CUSTOMER_CODE,Year,MONTH,ATTRIBUTE,Value,Month,Day,Date
3,0000F727854B50BB95C054B39C1FE5,2022,T11,Net Cash,500000,11,1,2022-11-01
97,000877FD1B07EB1646272C0BF682B5,2022,T10,Net Cash,1100000,10,1,2022-10-01
101,000877FD1B07EB1646272C0BF682B5,2022,T11,Net Cash,1120000,11,1,2022-11-01
105,000877FD1B07EB1646272C0BF682B5,2022,T12,Net Cash,-2517546,12,1,2022-12-01
175,0020100DBC05FEB6E5FAAE2D0FF6C5,2022,T10,Net Cash,5100000,10,1,2022-10-01


In [377]:
Transaction_history_Net_cash = Transaction_history_Net_cash.rename(columns={'Value':'NetCash'})

In [378]:
Historical_details_final['Date'] = Historical_details_final['Date'].astype('datetime64[ns]')

In [379]:
Historical_details_final = Historical_details_final.merge(Transaction_history_Net_cash[['CUSTOMER_CODE','Date','NetCash']], how='left', left_on = ['CUSTOMER_CODE','Date'], right_on = ['CUSTOMER_CODE','Date'])

In [380]:
Historical_details_final['NAV_exclude_cash'] = Historical_details_final['NAV'] - Historical_details_final['Cash']

In [381]:
Historical_details_final.sort_values('Date')
Historical_details_final['Previous_NAV']= Historical_details_final.groupby('CUSTOMER_CODE')['NAV'].transform(lambda x: x.shift(1))
Historical_details_final.head()

,CUSTOMER_CODE,Date,NAV,Cash,Max_NAV,Max_Cash,Latest_Cash,Latest_NAV,%_cash_over_max_cash,%_NAV_over_max_NAV,...,AGE,CUSTOMER_PRODUCT_TYPE,WEALTH_RANGE_NAME,CUSTOMER STATUS,Open_Date,Month_on_board,Web_trade_focus_FLG,NetCash,NAV_exclude_cash,Previous_NAV
0,0000F727854B50BB95C054B39C1FE5,2022-11-01,500001.294117,500001.294118,500001.294117,500001.294118,449125.45,449125.45,1.0,1.0,...,42.0,Not yet trading,W2:(0-10Mn),Dummy,2022-11-08 00:00:00,1.0,NaN,500000,-0.000001,NaN
1,0000F727854B50BB95C054B39C1FE5,2022-12-01,492823.318181,492823.318182,500001.294117,500001.294118,449125.45,449125.45,0.985644,0.985644,...,42.0,Not yet trading,W2:(0-10Mn),Dummy,2022-11-08 00:00:00,2.0,NaN,NaN,-0.000001,500001.294117
2,0000F727854B50BB95C054B39C1FE5,2023-01-01,484652.8125,484652.8125,500001.294117,500001.294118,449125.45,449125.45,0.969303,0.969303,...,42.0,Not yet trading,W2:(0-10Mn),Dummy,2022-11-08 00:00:00,3.0,NaN,NaN,0.0,492823.318181
3,0000F727854B50BB95C054B39C1FE5,2023-02-01,475441.35,475441.35,500001.294117,500001.294118,449125.45,449125.45,0.95088,0.95088,...,42.0,Not yet trading,W2:(0-10Mn),Dummy,2022-11-08 00:00:00,4.0,NaN,NaN,0.0,484652.8125
4,0000F727854B50BB95C054B39C1FE5,2023-03-01,466438.695652,466438.695652,500001.294117,500001.294118,449125.45,449125.45,0.932875,0.932875,...,42.0,Not yet trading,W2:(0-10Mn),Dummy,2022-11-08 00:00:00,5.0,NaN,NaN,-0.0,475441.35


In [382]:
Historical_details_final['Delta_NAV_exclude_cash'] = Historical_details_final['NAV'] - Historical_details_final['Previous_NAV'] - Historical_details_final['NetCash']

In [383]:
Historical_details_final['%_gain_loss'] = np.divide(Historical_details_final['Delta_NAV_exclude_cash'],  Historical_details_final['Previous_NAV'], where = Historical_details_final['Previous_NAV'] != 0)

In [384]:
def trendline(x, y, order=1):
    return np.polyfit(x, y, order)[-2]

NAV_trend = Historical_details_final.groupby('CUSTOMER_CODE').apply(lambda subdf: trendline(subdf['Month_on_board'].astype('float').values, subdf['NAV'].astype('float').values))

C:\Users\DUONG\AppData\Local\Temp\ipykernel_9328\2784942160.py:4: RankWarning:

Polyfit may be poorly conditioned

C:\Users\DUONG\AppData\Local\Temp\ipykernel_9328\2784942160.py:4: RankWarning:

Polyfit may be poorly conditioned

C:\Users\DUONG\AppData\Local\Temp\ipykernel_9328\2784942160.py:4: RankWarning:

Polyfit may be poorly conditioned

C:\Users\DUONG\AppData\Local\Temp\ipykernel_9328\2784942160.py:4: RankWarning:

Polyfit may be poorly conditioned

C:\Users\DUONG\AppData\Local\Temp\ipykernel_9328\2784942160.py:4: RankWarning:

Polyfit may be poorly conditioned

C:\Users\DUONG\AppData\Local\Temp\ipykernel_9328\2784942160.py:4: RankWarning:

Polyfit may be poorly conditioned

C:\Users\DUONG\AppData\Local\Temp\ipykernel_9328\2784942160.py:4: RankWarning:

Polyfit may be poorly conditioned

C:\Users\DUONG\AppData\Local\Temp\ipykernel_9328\2784942160.py:4: RankWarning:

Polyfit may be poorly conditioned

C:\Users\DUONG\AppData\Local\Temp\ipykernel_9328\2784942160.py:4: RankWarning:



C:\Users\DUONG\AppData\Local\Temp\ipykernel_9328\2784942160.py:4: RankWarning:

Polyfit may be poorly conditioned

C:\Users\DUONG\AppData\Local\Temp\ipykernel_9328\2784942160.py:4: RankWarning:

Polyfit may be poorly conditioned

C:\Users\DUONG\AppData\Local\Temp\ipykernel_9328\2784942160.py:4: RankWarning:

Polyfit may be poorly conditioned

C:\Users\DUONG\AppData\Local\Temp\ipykernel_9328\2784942160.py:4: RankWarning:

Polyfit may be poorly conditioned

C:\Users\DUONG\AppData\Local\Temp\ipykernel_9328\2784942160.py:4: RankWarning:

Polyfit may be poorly conditioned

C:\Users\DUONG\AppData\Local\Temp\ipykernel_9328\2784942160.py:4: RankWarning:

Polyfit may be poorly conditioned

C:\Users\DUONG\AppData\Local\Temp\ipykernel_9328\2784942160.py:4: RankWarning:

Polyfit may be poorly conditioned

C:\Users\DUONG\AppData\Local\Temp\ipykernel_9328\2784942160.py:4: RankWarning:

Polyfit may be poorly conditioned

C:\Users\DUONG\AppData\Local\Temp\ipykernel_9328\2784942160.py:4: RankWarning:



In [385]:
NAV_trend = pd.DataFrame(NAV_trend)

In [386]:
Historical_details_final = Historical_details_final.merge(NAV_trend, how='left', left_on='CUSTOMER_CODE', right_on='CUSTOMER_CODE')

In [387]:
Historical_details_final = Historical_details_final.rename(columns={0:'Beta'})

In [388]:
Historical_details_final.columns

Index(['CUSTOMER_CODE', 'Date', 'NAV', 'Cash', 'Max_NAV', 'Max_Cash',
       'Latest_Cash', 'Latest_NAV', '%_cash_over_max_cash',
       '%_NAV_over_max_NAV', '%_Latest_Cash_over_max_cash',
       '%_Latest_NAV_over_max_NAV', '%_Cash_over_NAV', 'MA_3M_NAV',
       'MA_3M_Cash', '%NAV_MA_3M_NAV', '%Cash_MA_3M_Cash', 'Stock_nunique',
       'Matched_value_sum', 'No_orders_sum', 'AGE', 'CUSTOMER_PRODUCT_TYPE',
       'WEALTH_RANGE_NAME', 'CUSTOMER STATUS', 'Open_Date', 'Month_on_board',
       'Web_trade_focus_FLG', 'NetCash', 'NAV_exclude_cash', 'Previous_NAV',
       'Delta_NAV_exclude_cash', '%_gain_loss', 'Beta'],
      dtype='object')

In [389]:
First_NAV_Customer = pd.DataFrame(Historical_details_final.groupby('CUSTOMER_CODE')['NAV'].first())

In [390]:
First_NAV_Customer = First_NAV_Customer.rename(columns={'NAV':'First_NAV'})

In [391]:
Historical_details_final = Historical_details_final.merge(First_NAV_Customer, how='left', left_on='CUSTOMER_CODE', right_on='CUSTOMER_CODE')

In [392]:
Historical_details_final['NAV_Trend_Indicator'] = np.divide(Historical_details_final['Beta'], Historical_details_final['First_NAV'], where = Historical_details_final['First_NAV'] != 0)

In [ ]:
Historical_details_final['%_NetCash_Prev_NAV'] = Historical_details_final['NetCash'] / Historical_details_final['Previous_NAV'] 

In [393]:
Historical_details_final.to_csv('Historical_details_final-v4.csv')